In [1]:
import numpy as np
import matplotlib.pyplot as plt

# # Define Domain
# # Space
# Xpts = int(1e2)  # Grid Points
# X0, Xf = 0, 10  # Domain
# Xlng = Xf-X0
# x = np.linspace(X0, Xf, num=Xpts, endpoint=False)  # Spatial Domain
# dx = Xlng/Xpts  # Grid Size

# # Time
# Tpts = int(1e2)  # Time Steps
# T0, Tf = 0, 2  # Domain
# Tlng = Tf-T0
# t = np.linspace(T0, Tf, num=Tpts, endpoint=False)
# dt = Tlng/Tpts  # Time Step Size

# Define Domain
# Space
Xpts = int(1e2)  # Grid Points
X0, Xf = 0, 10  # Domain
Xlng = Xf-X0
dx = Xlng/Xpts  # Grid Size
x = np.arange(X0, Xf, dx)  # Spatial Domain

# Time
dt = 2e-4  # Time Step Size
T0, Tf = 0, 10  # Domain
Tlng = Tf-T0
Tpts = int(Tlng/dt)  # Time Steps
t = np.linspace(T0, Tf, num=Tpts, endpoint=False)

# Numerical Parameters
xx, tt = np.meshgrid(x,t, sparse=False, indexing='xy') # Spatial-Temporal Domain
lmbd = dt / dx # 1e2/2.5e2 = .4
n_0 = 3 / (4 * np.pi)
Gamma_0 = 1 # input("Enter Gamma_0: ")
kappa_0 = 1 # input("Enter kappa_0: ")
beta = 1

# Correlation Parameters
k_fft_norm = 2*np.pi/(Xpts*dx)
k = k_fft_norm * np.linspace(-Xpts/2, Xpts/2-1,Xpts)  # Fourier Domain
x3 = np.linspace(-Xlng, 2*Xlng, 3*Xpts-2)  # Correlation Domain

print(lmbd)

0.002


In [2]:
def memoryPDE(u_IC):
    u = np.zeros((2,Tpts,Xpts))
    u[:,0] = np.copy(u_IC)
    Fu = np.zeros((2,Tpts,Xpts))
    return u, Fu

In [3]:
# Initial Conditions
disp_freq = 3 * 2 * np.pi / Xlng
n_IC = n_0*np.ones(Xpts) + .1*np.sin(disp_freq * x)
v_IC = 1+.1 * np.sin(disp_freq * x)
# v_IC = np.zeros(Xpts)
# v_IC[0:int(Xpts/2)] = 1 
# n_IC = n_0 * np.ones(Xpts) + .0001 * (np.random.rand(Xpts))
# v_IC = .0001 * (np.random.rand(Xpts))

# Memory Allocation
n, Fn = memoryPDE(n_IC)
v, Fv = memoryPDE(v_IC)
phi = np.zeros((2,Tpts,Xpts))
rhs = np.zeros((2,Tpts,Xpts))
godunov_flux = np.zeros((2,2,Tpts,Xpts)) #eq,c,t,x

In [4]:
# =============== #
# Solve Functions #
# =============== #
def solve_phic(cor, cur_time, un, uphi):
    uphi_c_cur = uphi[cor,cur_time]
    un_c_cur = un[cor,cur_time]
    A = np.zeros(Xpts)
    # Define b
    b = 3 - 4 * np.pi * dx * dx * un_c_cur
    b = b - np.mean(b)
    # First sweep
    A[0] = -0.5
    b[0] = -0.5 * b[0]
    for ii in range(1, Xpts):
        A[ii] = -1 / (2 + A[ii - 1])
        b[ii] = (b[ii - 1] - b[ii]) / (2 + A[ii - 1])
    # Second sweep
    uphi_c_cur[0] = b[Xpts - 1] - b[Xpts - 2]
    for ii in range(1, Xpts - 1):
        uphi_c_cur[ii] = (b[ii - 1] - uphi_c_cur[ii - 1]) / A[ii - 1]
    return uphi_c_cur

def nonisotropic_correlations(unc, cur_time, n3, x, x3,f_corr):
    conc = unc / n_0
    Gamma = Gamma_0 * conc ** (1 / 3)
    kappa = kappa_0 * conc ** (1 / 6)

    n3[0:N] = unc[cur_time]
    n3[N:2 * N] = unc[cur_time]
    n3[2 * N:3 * N] = unc[cur_time]
    for jj in range(N):
        rho_int = - 2 * np.pi * Gamma[jj] * np.exp(- kappa[jj] * np.abs(x3 - x[jj])) / kappa[jj]
        f_corr[jj] = dx * np.sum(n3 * rho_int)
    return f_corr

def fft_meanfield(k,nc,Gamma, kappa):
    delta_n = nc - n_0
    def dcf(k, Gamma, kappa):
        return 4 * np.pi * Gamma / (k ** 2 + kappa ** 2)

    dcfunc = dcf(k,Gamma,kappa)
    fhat = np.fft.fftshift(np.fft.fft(delta_n))
    conv = fhat * dcfunc
    conv = np.fft.ifft(np.fft.ifftshift(conv))
    conv = np.real(conv)
    return conv

def fft_meyerf(k,nc,Gamma, kappa, beta):
    delta_n = nc - n_0

    # f_fft_norm = 1 / dx
    # k_fft_norm = 2 * np.pi / (N * dx)

    # Parameters
    Nr = int(1e3)
    rmax = 100  # TODO: Change per loop
    r = np.linspace(0, rmax, Nr)

    dcf = np.exp(-beta * r ** 2)
    dcf_fft = np.fft.fftshift(np.fft.fft(dcf))
    dcf_fft_ex = (np.pi / beta) ** (3 / 2) * np.exp(- k ** 2 / (4 * beta))

    n_hat = np.fft.fftshift(np.fft.fft(delta_n))
    conv = n_hat * dcf_fft
    conv = np.fft.ifft(np.fft.ifftshift(conv))
    conv = np.real(conv)
    return conv

In [ ]:
# ===== #
# Solve #
# ===== #
# for c in range(2): # Iterate through correlations
    
c=0
for tt in range(1,Tpts): # Iterate Time
    for ii in range(Xpts):
        Fn[c,tt,ii] = n[c,tt-1,ii]*v[c,tt-1,ii]
        Fv[c,tt,ii] = .5*v[c,tt-1,ii]*v[c,tt-1,ii]+np.log(n[c,tt-1,ii])

    vec = np.stack((n,v))
    vec_L = np.roll(vec, 1,axis=-1)
    vec_flux = np.stack((Fn, Fv))
    vec_flux_L = np.roll(vec_flux, 1,axis=-1)

    for eq in range(2): # Iterate over system
        for ii in range(Xpts): # Iterate over spacial values
            if vec_L[eq,c,tt,ii] > vec[eq,c,tt,ii]:
                godunov_flux[eq,c,tt,ii] = np.maximum(vec_flux_L[eq,c,tt,ii], vec_flux[eq,c,tt,ii])
            elif vec_L[eq,c,tt,ii] < vec[eq,c,tt,ii]:
                godunov_flux[eq,c,tt,ii] = np.minimum(vec_flux_L[eq,c,tt,ii], vec_flux[eq,c,tt,ii])
            else:
                godunov_flux[eq,c,tt,ii] = 0.0

    phi[c,tt] = solve_phic(c,tt,n,phi)

#         if c = 0:
#             # f_corr = anisotropic_correlations(n,n3,x,x3,f_corr)
#             f_corr = fft_meanfield(k,n,Gamma_0,kappa_0)
#             rhs = -(f_corrR - f_corrL)/dx - Gamma_0*(phi[c,tt] - np.roll(phi[c,tt],1))/dx # TODO: -1/2 or 1??
#         else:
#             rhs[c,tt] = -Gamma_0*(phi[c,tt] - np.roll(phi[c,tt],1))/dx

    rhs[c,tt] = -Gamma_0*(phi[c,tt] - np.roll(phi[c,tt],1))/dx

    n[c,tt,0] = n[c,tt-1,0]-lmbd*(Fn[c,tt-1,0]-Fn[c,tt-1,Xpts-1])
    v[c,tt,0] = v[c,tt-1,0]-lmbd*(Fv[c,tt-1,0]-Fv[c,tt-1,Xpts-1])+rhs[c,tt,0]
    for ii in range(1,Xpts):
        n[c,tt,ii] = n[c,tt-1,ii]-lmbd*(Fn[c,tt-1,ii]-Fn[c,tt-1,ii-1])
        v[c,tt,ii] = v[c,tt-1,ii]-lmbd*(Fv[c,tt-1,ii]-Fv[c,tt-1,ii-1])+rhs[c,tt,ii]

    if  tt % 50 == 0:
        print(tt)
#         if tt < 10 and c == 0:
#             print(tt)
#             plt.plot(x,n[c,tt],label="n")
#             plt.plot(x,v[c],label="v")
#             plt.show(block=False)

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500
9550
9600
9650
9700
9750
9800
9850
9900
9950
10000
10050
10100
10150
1

In [ ]:
# ============== #
# Plot Functions #
# ============== #

snaps = 20

def plot(ux, u):
    for c in range(2):
        fig = plt.figure(figsize=(15,15))
        plt.title('Γ = ' + str(Gamma_0) + ', κ = ' + str(kappa_0))
        for tt in range(Tpts):
            if tt % (Tpts / snaps) == 0:
                plt.plot(ux, u[c,tt], label=str(tt/snaps))
        plt.legend()
    plt.show(block=False)
    
#     plt.savefig(.png)

def cmap(ux, ut, u): 
    for c in range(2):
        fig = plt.figure(figsize=(15,15))
#         color_map = plt.contourf(ux, ut, u[c,:])
        color_map = plt.imshow(u[c,:], cmap='viridis', origin='lower', extent=(X0,Xf,T0,Tf), aspect='auto')
        plt.title('Γ = ' + str(Gamma_0) + ', κ = ' + str(kappa_0))
        plt.colorbar()
        plt.ylabel("Time")
        plt.xlabel("Space")
        plt.show(block=False)    

def calculate_2dft(u):
    fft = np.fft.ifftshift(u-np.mean(u[:,:]))
    fft = np.fft.fft2(fft)
    return np.abs(np.fft.fftshift(fft))

def disp_rel_cmap(ux, ut, u):
    for c in range(2):
        fft = calculate_2dft(u[c])
        fig = plt.figure(figsize=(15,15))
        color_map = plt.contourf(ux, ut, fft)
        color_map = plt.imshow(fft, cmap='viridis', origin='lower', extent=(X0,Xf,T0,Tf), aspect='auto')
#         plt.title('Γ = ' + str(Gamma[ii]) + ', κ = ' + str(kappa[jj]))
        plt.title('Γ = ' + str(Gamma_0) + ', κ = ' + str(kappa_0))
        plt.colorbar()
        plt.ylabel("Time")
        plt.xlabel("Space")
        plt.show(block=False)
        
def plot3D():
    fig = plt.figure()
    

In [ ]:
# ==== #
# Plot #
# ==== #
# n_tot_rev = np.flip(n_tot,0)
# v_tot_rev = np.flip(v_tot,0)

# plot(x[0:10],n[:,:,0:10])
print(lmbd)
plot(x,n)
plot(x,v)
cmap(x, t, n) # TODO: no rev?? check orientation of colormap wrt time axis
disp_rel_cmap(x, t, n)

In [ ]:

def subplot(u):
    fig, axes = plt.subplots(nrows=3, ncols=3)
    # find minimum of minima & maximum of maxima
    minmin = np.min(uc for uc in u)
    maxmax = np.max(uc for uc in u)
    for c in range(2):
        images = []
        for ii in range(3):
            for jj in range(3):
                im = axes[ii][jj].imshow(u[ii+jj], vmin=minmin, vmax=maxmax,
                                         extent=(X0,Xf,T0,Tf), aspect='auto', cmap='viridis')
                axes[ii][jj].set_title('Γ = ' + str(Gamma[ii]) + ', κ = ' + str(kappa[jj]))
                axes[ii][jj].set_ylabel("Time")
                axes[ii][jj].set_xlabel("Space")
                images.append(im)

        fig.tight_layout(pad = .01)
        fig.subplots_adjust(top=0.9)
        fig.suptitle("Density: Xptso Correlations")
        cbar = fig.colorbar(im, ax=axes.ravel().tolist(), shrink=1)
        plt.show(block=False)
    